# How to solve 90% of NLP problems: a step-by-step guide

While there exist a wealth of elaborate and abstract NLP techniques, clustering and classification should always be in our toolkit as the first techniques to use when dealing with this kind of data. In addition to being amongst some of the easiest to scale in production, their ease of use can quickly help business address a set of applied problems:

* How do you automatically make the distinction between different categories of sentences?
* How can you find sentences in a dataset that are most similar to a given one?
* How can you extract a rich and concise representation that can then be used for a range of other tasks?
* Most importantly, how do you find quickly whether these tasks are possible on your dataset at all?

## Our Dataset: Disasters on social media
Contributors looked at over 10,000 tweets retrieved with a variety of searches like “ablaze”, “quarantine”, and “pandemonium”, then noted whether the tweet referred to a disaster event (as opposed to a joke with the word or a movie review or something non-disastrous). 
[Download](https://www.crowdflower.com/data-for-everyone/)
search: Disasters on social media

## Why it matters
We will try to correctly predict tweets that are about disasters. This is a very relevant problem, because:

* It is actionable to anybody trying to get signal from noise (such as police departments in this case)
* It is tricky because relying on keywords is harder than in most cases like spam